# 输出修正解析器

该输出解析器包装了另一个输出解析器，并尝试修复任何错误。

Pydantic 出于安全考虑尝试解析 LLM 响应。如果解析不正确，则会出现错误。

但我们除了抛出错误之外还可以做其他事情。具体而言，我们可以将格式不正确的输出以及格式化指令一起传递给模型，并要求其进行修复。

对于这个示例，我们将使用上面的 OutputParser。如果我们将格式不符合模式的结果传递给它，会发生以下情况：

In [3]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [4]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
        
actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=Actor)

In [5]:
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}"

In [6]:
parser.parse(misformatted)

OutputParserException: Failed to parse Actor from completion {'name': 'Tom Hanks', 'film_names': ['Forrest Gump']}. Got: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

现在我们可以构建并使用一个 `OutputFixingParser`。该输出解析器需要另一个输出解析器和一个 LLM 作为参数，用于尝试纠正任何格式错误。

In [7]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

In [8]:
new_parser.parse(misformatted)

Actor(name='Tom Hanks', film_names=['Forrest Gump'])